# FIDE Data on player ranking
an in-depth analysis

https://github.com/lichess-org/berserk/blob/master/docs/usage.rst

In [12]:
api_token = "lip_boGM1MWUY1PXESFGzopR"
import berserk

In [13]:
session = berserk.TokenSession(api_token)
client = berserk.Client(session)

In [14]:
person_of_interest = "DrVitman"

In [28]:
import datetime

start = datetime.datetime.now()
#                       year,   month,  day
end = datetime.datetime(2024,   2,      1)

In [29]:
client.games.export_by_player(person_of_interest, start, end, max=100)

<generator object Games.export_by_player at 0x000001A1BC6FD8A0>

In [30]:
games = list(_)

In [33]:
games[0].split('\n')

['[Event "GM DrVitman simul"]',
 '[Site "https://lichess.org/Wcf0QCQ8"]',
 '[Date "2024.02.25"]',
 '[White "arsenalfanrichi"]',
 '[Black "DrVitman"]',
 '[Result "0-1"]',
 '[UTCDate "2024.02.25"]',
 '[UTCTime "00:39:04"]',
 '[WhiteElo "2142"]',
 '[BlackElo "2681"]',
 '[BlackTitle "GM"]',
 '[Variant "From Position"]',
 '[TimeControl "300+0"]',
 '[ECO "?"]',
 '[Termination "Normal"]',
 '[FEN "rnbqkb1r/p2ppppp/5n2/1ppP4/2P5/8/PP2PPPP/RNBQKBNR w KQkq - 0 1"]',
 '[SetUp "1"]',
 '',
 '1. cxb5 a6 2. bxa6 g6 3. Nc3 Bg7 4. e4 O-O 5. h3 d6 6. Nf3 Bxa6 7. Bxa6 Nxa6 8. O-O Nc7 9. Bf4 Qd7 10. Qe2 Rfb8 11. Rfd1 Nb5 12. Nxb5 Rxb5 13. b3 Rba5 14. Nd2 Rxa2 15. Rxa2 Rxa2 16. Qe3 Qb5 17. Nc4 Nd7 18. e5 dxe5 19. Nxe5 Nxe5 20. Bxe5 Re2 0-1']

In [35]:
games[-1].split('\n')

['[Event "GM DrVitman simul"]',
 '[Site "https://lichess.org/sez0RuNm"]',
 '[Date "2024.02.14"]',
 '[White "DrVitman"]',
 '[Black "AmirchikHab123"]',
 '[Result "1-0"]',
 '[UTCDate "2024.02.14"]',
 '[UTCTime "14:23:16"]',
 '[WhiteElo "2681"]',
 '[BlackElo "1237"]',
 '[WhiteTitle "GM"]',
 '[Variant "Standard"]',
 '[TimeControl "300+0"]',
 '[ECO "C64"]',
 '[Termination "Time forfeit"]',
 '',
 '1. e4 e5 2. Nf3 Nc6 3. Bb5 Bc5 4. O-O a6 5. Bxc6 bxc6 6. Nxe5 Nf6 7. d4 Bb6 8. Qd3 d6 9. Nc4 d5 10. exd5 cxd5 11. Nxb6 cxb6 12. Re1+ Be6 13. Bg5 h6 14. Bh4 O-O 15. Nd2 Qe7 16. Nf3 Ne4 17. Bxe7 1-0']

In [38]:
print([game.split("\n") for game in games])

[['[Event "GM DrVitman simul"]', '[Site "https://lichess.org/Wcf0QCQ8"]', '[Date "2024.02.25"]', '[White "arsenalfanrichi"]', '[Black "DrVitman"]', '[Result "0-1"]', '[UTCDate "2024.02.25"]', '[UTCTime "00:39:04"]', '[WhiteElo "2142"]', '[BlackElo "2681"]', '[BlackTitle "GM"]', '[Variant "From Position"]', '[TimeControl "300+0"]', '[ECO "?"]', '[Termination "Normal"]', '[FEN "rnbqkb1r/p2ppppp/5n2/1ppP4/2P5/8/PP2PPPP/RNBQKBNR w KQkq - 0 1"]', '[SetUp "1"]', '', '1. cxb5 a6 2. bxa6 g6 3. Nc3 Bg7 4. e4 O-O 5. h3 d6 6. Nf3 Bxa6 7. Bxa6 Nxa6 8. O-O Nc7 9. Bf4 Qd7 10. Qe2 Rfb8 11. Rfd1 Nb5 12. Nxb5 Rxb5 13. b3 Rba5 14. Nd2 Rxa2 15. Rxa2 Rxa2 16. Qe3 Qb5 17. Nc4 Nd7 18. e5 dxe5 19. Nxe5 Nxe5 20. Bxe5 Re2 0-1'], ['[Event "GM DrVitman simul"]', '[Site "https://lichess.org/kb4PnOKe"]', '[Date "2024.02.25"]', '[White "Delloo1302"]', '[Black "DrVitman"]', '[Result "0-1"]', '[UTCDate "2024.02.25"]', '[UTCTime "00:39:04"]', '[WhiteElo "1633"]', '[BlackElo "2681"]', '[BlackTitle "GM"]', '[Variant "Fr